In [2]:
import os
print(os.environ['CONDA_DEFAULT_ENV'])

KeyError: 'CONDA_DEFAULT_ENV'

In [2]:
import torch
import numpy as np

In [4]:
x = torch.tensor([1.,2,3])
w = torch.tensor([4.,5,6],requires_grad = True)
b = torch.tensor(5.,requires_grad = True)
y = torch.dot(w,x) + b

In [10]:
y = torch.dot(w,x) + b
y


tensor(37., grad_fn=<AddBackward0>)

In [11]:
y.backward()

In [12]:
print(w.grad)

tensor([2., 4., 6.])


### Broadcasting_test


In [38]:
# x = torch.randn(5,2)
# y = torch.rand(2)
# print(x)
# print(y)
# x + y
x = np.full((61,),1)
np.broadcast_to(x, (1760,61))


array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [14]:
xx = torch.randn(10000,3)
torch.sum(torch.nn.functional.softmax(xx,dim = 1),dim = 1)

tensor(False)

# Linear Regression


In [9]:

inputs = np.array([[73,67,43],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                  [69,96,70]],dtype = 'float32')

# 5 (samples) * 3 (features)

In [10]:
targets = np.array([[56,70],[81,101],[119,133],[22,37],[103,119]],dtype  = 'float32')

# 5 (samples) * 2 (outputs)

In [11]:
### convert to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [12]:
w = torch.randn(2,3,requires_grad = True)
b = torch.randn(2,requires_grad = True)

print(w)
print(b)

tensor([[ 0.5002, -0.9489, -0.9359],
        [-0.8882, -2.7106, -0.2473]], requires_grad=True)
tensor([-1.3855, -1.0733], requires_grad=True)


In [13]:
def model(x):
    return x @ w.t() + b

def mse(a,b):
    diff = a - b
    return torch.sum(diff*diff)/ diff.numel()

In [14]:
preds = model(inputs)
loss = mse(targets,preds)
loss

tensor(108930.2734, grad_fn=<DivBackward0>)

In [15]:
for i in range(100):
    preds = model(inputs)
    loss = mse(targets,preds)
    
    if loss > 0:
        loss.backward()
        with torch.no_grad():
            w -= w.grad * 1e-4
            b -= b.grad * 1e-4
            w.grad.zero_()
            b.grad.zero_()

In [16]:
preds = model(inputs)
loss = mse(targets,preds)
loss

tensor(8.8847, grad_fn=<DivBackward0>)

In [17]:
preds

tensor([[ 57.2844,  70.0495],
        [ 79.6100, 101.6694],
        [124.4033, 131.0999],
        [ 22.8508,  36.6573],
        [ 96.0971, 120.8757]], grad_fn=<AddBackward0>)

 # Linear Regression using pytorch built ins
 

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [20]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(1,5, requires_grad=True)
print(input.shape)

target = x = torch.tensor([1])
print(target.shape)
output = loss(input, target)
output.item()

torch.Size([1, 5])
torch.Size([1])


0.8033408522605896

In [19]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [20]:
train_ds = TensorDataset(inputs, targets)
train_dl = DataLoader(train_ds, 2, shuffle=True)

In [21]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2161,  0.1602, -0.3883],
        [ 0.4674,  0.3618, -0.0845]], requires_grad=True)
Parameter containing:
tensor([0.1010, 0.1536], requires_grad=True)


In [22]:
loss_fn = F.mse_loss
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [23]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            preds = model(xb)
            loss = loss_fn(preds,yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [24]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 6.4405
Epoch [20/100], Loss: 211.7910
Epoch [30/100], Loss: 17.0926
Epoch [40/100], Loss: 68.8298
Epoch [50/100], Loss: 65.8438
Epoch [60/100], Loss: 0.8004
Epoch [70/100], Loss: 11.0861
Epoch [80/100], Loss: 3.9161
Epoch [90/100], Loss: 41.3116
Epoch [100/100], Loss: 2.3204
